# 기상데이터 전처리 및 결측치 처리

### 결측치 처리 과정에서 알아봐야 할 것
- 월일시 데이터 쪼개기
- 0으로 채워야하는 변수는 단순하게 채울것
- 실제로 일조량, 일사량은 햇빛에 의해서 발생하므로 실제 해가 떠있는 시간에 결측치가 없는지 확인
- 결측치가 많지 않다면 제거해서 하는것도 괜찮아보임
- 하지만 결측치가 많다면 변수를 제외시키거나 다른 방법을 찾아야함

### 최초진단
- 날씨를 나타내는 ww변수는 일단 제외시키고 진행하는 것을 권고
- 다양하게 활용할 수 있다면 추가시키고 제어시켜서 진행하는것도 괜찮아 보임
- 일사량과 일조량의 차이가 존재
    - 일사량: 태양열에 의해 지면에서 올라오는 빛 
    - 일조량: 태양으로부터 직접적으로 맞닿는 빛 초단위로 측정됨
    - 태양광 산업에서는 일조 시간의 편차를 체크하는 것을 주요 요소로 판단
    - 우리가 진행하는 데이터도 일조 시간과 관련해서 따져보면 나쁘지 않을듯.
    
### 코드 구조
- 웬만해선 테스트 셋에도 똑같이 적용해야 하기 때문에 함수를 생성하고 그에 맞게 대입하는 방법을 주로 하는 것이 좋겠다

### 일출 일몰
- 황해안에서 동해안까지 차이는 10분 안쪽으로 차이가난다. -> 지역별 차이가 크지 않다.
- 시간대별 데이터가 있기 때문에 일자별 일출 및 일몰 시간대를 구분할 수 있겠다.
- 이것을 이용하여 일출 ~ 일몰때의 결측치와 일몰 ~ 일출 때의 결측치를 별도로 다룰 수 있겠다.
- 데이터 출처 : https://astro.kasi.re.kr/index
- 한국천문연구원 천문우주지식센터
- 2022년 서울특별시기준 시간 단위
- 2월 29일은 가장 가까운 2020년 데이터 활용

## 함수형 코드 구조

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data
train = pd.read_csv('../data/train.csv')
sun = pd.read_csv('../data/sun.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
# 결측치 숫자를 NaN으로 만들어야함
def makenum(df, numbers):
    for colname in df.columns:
        df[colname][df[colname].isin(numbers)] = np.nan
    return df

In [4]:
# 열 이름마다 들어있는 surface_tp_train. 제거
def change_colnames(df, string):
    df.columns = list(pd.DataFrame(df.columns)[0].str.replace(string, ''))
    return df

In [5]:
# 0으로 채워야 하는 변수는 모두 채워버리기
def nanfill_select(df, list):
    df[list] = df[list].fillna(0)
    return df

In [6]:
# 월일시 데이터 분리
def splitmmddhh(df, datecolumn):
    df[datecolumn] = df[datecolumn].astype(str).str.zfill(6)
    df['mm'] = df[datecolumn].str.slice(0, 2).astype(int)
    df['dd'] = df[datecolumn].str.slice(2, 4).astype(int)
    df['hh'] = df[datecolumn].str.slice(4, 6).astype(int)
    return df.drop(datecolumn, axis = 1)

## 적용형 코드 구조

## Train

In [7]:
# 결측 숫자 NaN
train2 = makenum(train, [-99.9, -99, -999])

In [8]:
# 열 이름 간략하게 바꾸기
train3 = change_colnames(train2, 'surface_tp_train.')

In [9]:
# 월, 일, 시 분리하기
train4 = splitmmddhh(train3, 'mmddhh')

In [10]:
# 데이터 합치기
train5 = pd.merge(train4, sun, how='left', on = ['mm', 'dd'])

In [12]:
# 결측치 빼지 말고 저장하기
train5.to_csv('../data/train_yestsnona.csv', index = False)

In [13]:
# 지면온도 결측치 제외(어차피 test에서는 없어서 test에서만 안돌리면 괜찮을듯)
train6 = train5[train5['ts'].notnull()]

In [15]:
# save data
train6.to_csv("../data/train_yests.csv", index = False)

==========================================================================================================

## test

In [17]:
# 결측 숫자 NaN
test2 = makenum(test, [-99.9, -99, -999])

In [18]:
# 열 이름 간략하게 바꾸기
test3 = change_colnames(test2, 'surface_tp_test.')

In [19]:
# 월, 일, 시 분리하기
test4 = splitmmddhh(test3, 'mmddhh')

In [20]:
# 데이터 합치기
test5 = pd.merge(test4, sun, how='left', on = ['mm', 'dd'])

In [21]:
# save data
test5.to_csv("../data/test_yests.csv", index = False)